In [ ]:
from dotenv import load_dotenv
import os

_BEXIO_ENV_LOADED = False

def get_bexio_api_key() -> str:
    global _BEXIO_ENV_LOADED
    if not _BEXIO_ENV_LOADED:
        load_dotenv()
        _BEXIO_ENV_LOADED = True
    api_key = os.getenv("BEXIO_API_KEY")
    if not api_key:
        raise RuntimeError("Missing BEXIO_API_KEY in environment variables.")
    return api_key

def get_bexio_headers() -> dict:
    return {
        'Accept': "application/json",
        'Content-Type': "application/json",
        'Authorization': f"Bearer {get_bexio_api_key()}",
    }


In [ ]:
import re
from typing import Dict, Any, Optional
import requests


def fetch_bexio_article_by_id(article_id: int) -> Dict[str, Any]:
    url = f"https://api.bexio.com/2.0/article/{article_id}"
    response = requests.get(url, headers=get_bexio_headers(), timeout=30)
    response.raise_for_status()
    data = response.json()
    return data


def get_article_description_html(article_id: int) -> str:
    article = fetch_bexio_article_by_id(article_id)
    return article.get("intern_description") or ""


def parse_description_html_to_pairs(description_html: str) -> Dict[str, str]:
    # Strip simple HTML tags and condense whitespace for key-value parsing
    text = re.sub(r"<[^>]+>", " ", description_html or "")
    text = re.sub(r"\s+", " ", text).strip()
    pairs: Dict[str, str] = {}

    # Best-effort patterns for optional fields commonly present in descriptions
    candidates = [
        ("HS-Code", r"(?:HS[-\s]?Code|HS)\s*[:=]\s*([0-9]{4,10})"),
        ("Zusatzcode", r"(?:Zusatzcode|ZS[-\s]?Code)\s*[:=]\s*([\w\-]{2,20})"),
        ("Shelf Life", r"(?:Shelf\s*Life|MHD|Best[-\s]?before)\s*[:=]\s*([0-9]{1,2}[.\-/][0-9]{1,2}[.\-/][0-9]{2,4})"),
        ("Unit", r"(?:Unit|Einheit)\s*[:=]\s*([A-Za-z0-9\s/().-]{1,40})"),
        ("Product code", r"(?:Product\s*code|Artikel[-\s]?Nr\.?|SKU|intern_code)\s*[:=]\s*([A-Za-z0-9\-_.]+)"),
        ("Name of Product", r"(?:Name\s*of\s*Product|Produktname)\s*[:=]\s*([A-Za-z0-9\s/().,\-+&]+)"),
    ]

    for key, pattern in candidates:
        m = re.search(pattern, text, flags=re.IGNORECASE)
        if m:
            pairs[key] = m.group(1).strip()

    return pairs


In [ ]:
from typing import List
import requests


def fetch_bexio_unit_name(unit_id: int) -> Optional[str]:
    url = f"https://api.bexio.com/2.0/unit/{unit_id}"
    response = requests.get(url, headers=get_bexio_headers(), timeout=30)
    if response.status_code != 200:
        return None
    try:
        data = response.json()
    except Exception:
        return None
    if isinstance(data, dict):
        return data.get("name")
    return None


def fetch_descriptions_for_products(product_ids: List[int]):
    results = []
    for product_id in product_ids:
        html = get_article_description_html(product_id)
        pairs = parse_description_html_to_pairs(html)
        results.append({
            "product_id": product_id,
            "intern_description_html": html,
            "description_pairs": pairs,
        })
    return results


In [1]:
print("hello")

hello


In [2]:
from dotenv import load_dotenv
load_dotenv()
import os
BEXIO_API_KEY = os.getenv("BEXIO_API_KEY")

In [2]:
BEXIO_API_KEY

'eyJhbGciOiJSUzI1NiIsInR5cCIgOiAiSldUIiwia2lkIiA6ICJkVTNTYXFLOHF1c25rakl4WEFsbE1EZk0zakRLYkJneDd3dlVVMHBsaUhFIn0.eyJleHAiOjE3NzU4MjE3NzMsImlhdCI6MTc2MDAxMDY2MSwianRpIjoiNDljZGE0ZGUtZTQ0NC00NDdmLWEwMmYtMjZhZDllNzQwZGQwIiwiaXNzIjoiaHR0cHM6Ly9hdXRoLmJleGlvLmNvbS9yZWFsbXMvYmV4aW8iLCJzdWIiOiI0OTQzYjIzNS1jOTYwLTExZTktYjE2Mi1hNGJmMDExY2U4NzIiLCJ0eXAiOiJCZWFyZXIiLCJhenAiOiJiZXhpb19wYXRfcHJvdmlkZXIiLCJzaWQiOiIxZDIzNTgyYi0zYzhiLTRmMTAtOThjMi0xZDBhNDM5M2IyODciLCJzY29wZSI6Im9wZW5pZCBhY2NvdW50aW5nIGtiX2RlbGl2ZXJ5X3Nob3cgYmFua19wYXltZW50X3Nob3cgcHJvamVjdF9lZGl0IGFjY291bnRpbmdfc2V0dGluZ3NfZWRpdCBrYl9jcmVkaXRfdm91Y2hlcl9zaG93IGtiX29mZmVyX2VkaXQgZmlsZSBwcm9qZWN0X3Nob3cgY29tcGFueV9wcm9maWxlIGtiX2JpbGxfc2hvdyBjb250YWN0X2VkaXQgbm90ZV9zaG93IGtiX2RlbGl2ZXJ5X2VkaXQga2JfZXhwZW5zZV9zaG93IHBheXJvbGxfcGF5c3R1Yl9zaG93IHBheXJvbGxfZW1wbG95ZWVfc2hvdyBrYl9hcnRpY2xlX29yZGVyX2VkaXQgYXJ0aWNsZV9lZGl0IGtiX2ludm9pY2VfZWRpdCBvZmZsaW5lX2FjY2VzcyBrYl9vcmRlcl9zaG93IGtiX2FydGljbGVfb3JkZXJfc2hvdyBwYXlyb2xsX3RpbWVfYWNjb3VudF9lZGl

In [3]:
#!/usr/bin/python
import requests

url = "https://api.bexio.com/3.0/currencies"

headers = {
    'Accept': "application/json",
    'Content-Type': "application/json",
    'Authorization': f"Bearer {BEXIO_API_KEY}",
}

response = requests.request("GET", url, headers=headers)

#print(response.text)


In [6]:
response.json()

[{'id': 1, 'name': 'CHF', 'round_factor': 0.05},
 {'id': 2, 'name': 'EUR', 'round_factor': 0.01},
 {'id': 3, 'name': 'USD', 'round_factor': 0.01},
 {'id': 4, 'name': 'GBP', 'round_factor': 0.01},
 {'id': 5, 'name': 'BRL', 'round_factor': 0.01},
 {'id': 6, 'name': 'JPY', 'round_factor': 0.01},
 {'id': 7, 'name': 'CNY', 'round_factor': 0.01},
 {'id': 8, 'name': 'SEK', 'round_factor': 0.01},
 {'id': 9, 'name': 'AUD', 'round_factor': 0.01},
 {'id': 10, 'name': 'CAD', 'round_factor': 0.01},
 {'id': 11, 'name': 'RUB', 'round_factor': 0.01},
 {'id': 12, 'name': 'DKK', 'round_factor': 0.01}]

In [1]:
url = "https://api.bexio.com/2.0/article/search"
payload = """
[
    {
        "field": "intern_code",
        "value": "30EM4AOPSc1030 ",
        "criteria": "="
    }
]
"""
headers = {
    'Accept': "application/json",
    'Content-Type': "application/json",
    'Authorization': f"Bearer {BEXIO_API_KEY}",
}

#response = requests.request("POST", url, data=payload, headers=headers)

NameError: name 'BEXIO_API_KEY' is not defined

In [8]:
payload = """
[
    {
        "field": "intern_code",
        "value": "80GY6AOPKc1012 ",
        "criteria": "="
    }
]
"""
response = requests.request("POST", url, data=payload, headers=headers)

In [ ]:
response.json()

[{'id': 136,
  'user_id': 1,
  'article_type_id': 1,
  'contact_id': 13,
  'deliverer_code': '',
  'deliverer_name': '',
  'deliverer_description': '',
  'intern_code': '80GY6AOPKc1012',
  'intern_name': 'Gruyère AOP 6M King Cuts',
  'intern_description': 'Gruy&egrave;re AOP 6M King Cuts<br />Unit:<br />Gross kg:&nbsp;<br />HS-Code: 0406.9099<br />Zusatzcode: 912',
  'purchase_price': None,
  'sale_price': None,
  'purchase_total': None,
  'sale_total': None,
  'currency_id': 1,
  'tax_income_id': 6,
  'tax_id': 6,
  'tax_expense_id': 10,
  'unit_id': None,
  'is_stock': False,
  'stock_id': None,
  'stock_place_id': None,
  'stock_nr': 0,
  'stock_min_nr': 0,
  'stock_reserved_nr': 0,
  'stock_available_nr': 0,
  'stock_picked_nr': 0,
  'stock_disposed_nr': 29320,
  'stock_ordered_nr': 0,
  'width': None,
  'height': None,
  'weight': None,
  'volume': None,
  'html_text': None,
  'remarks': '',
  'delivery_price': None,
  'article_group_id': 2,
  'account_id': None,
  'expense_accoun

In [ ]:
#!/usr/bin/python
import requests
unit_id=3
url = f"https://api.bexio.com/2.0/unit/{unit_id}"

headers = {
    'Accept': "application/json",
    'Content-Type': "application/json",
    'Authorization': f"Bearer {BEXIO_API_KEY}",
}

response = requests.request("GET", url, headers=headers)

print(response.text)


{"id":3,"name":"Kg"}


[{'id': 1019, 'document_nr': '251126_28', 'title': 'Order 28', 'contact_id': 83, 'contact_sub_id': None, 'user_id': 1, 'project_id': None, 'logopaper_id': 1, 'language_id': 4, 'bank_account_id': 1, 'currency_id': 1, 'payment_type_id': 4, 'header': '<span><strong>Loading Date:</strong> 26.11.2025</span><br /><strong>Order #:&nbsp;</strong>28<br /><span><strong>Order Date:</strong>&nbsp;20.11</span>.2025<br /><strong>Currency:&nbsp;</strong>CHF<br /><strong>VAT:</strong>&nbsp;FI10326282<br /><strong>EORI Nr.:&nbsp;</strong>FI1032628-2', 'footer': '<span>Preferential origin declaration:</span>&nbsp;The exporter of the products covered by this document declares that, except where otherwise clearly indicated, these products is of Switzerland preferential origin.<br /><br /><strong>"Steuerfreie Lieferung nach &sect;6 a UStG. Innergemeinschaftliche Lieferung / Verbringung von &Ouml;sterreich nach Finnland"</strong><br /><br />Best regards,<br />REAL SWISS CHEESE GMBH<br /><br /><em>Complaints

In [1]:
print(BEXIO_API_KEY)

NameError: name 'BEXIO_API_KEY' is not defined

In [8]:
import requests
url = "https://api.bexio.com/3.0/taxes"
response = requests.request("GET", url, headers=headers)


In [9]:
response.json()

[{'id': 3,
  'uuid': 'd029e3ad-db1a-11e9-ba9d-a4bf01195ac3',
  'name': 'lib.model.tax.ch.sales_export.name',
  'code': 'Export',
  'digit': '220',
  'type': 'not_taxable_turnover',
  'account_id': 127,
  'tax_settlement_type': 'none',
  'value': 0,
  'net_tax_value': None,
  'start_year': 2017,
  'end_year': 2026,
  'is_active': True,
  'display_name': 'Export - Export/Exempt 0.00%',
  'start_month': 12,
  'end_month': 12},
 {'id': 4,
  'uuid': 'd029e6fd-db1a-11e9-ba9d-a4bf01195ac3',
  'name': 'lib.model.tax.ch.sales_abroad.name',
  'code': 'ULA',
  'digit': '221',
  'type': 'not_taxable_turnover',
  'account_id': 127,
  'tax_settlement_type': 'none',
  'value': 0,
  'net_tax_value': None,
  'start_year': None,
  'end_year': None,
  'is_active': False,
  'display_name': 'ULA - Leistungen im Ausland 0.00%',
  'start_month': None,
  'end_month': None},
 {'id': 5,
  'uuid': 'd029e843-db1a-11e9-ba9d-a4bf01195ac3',
  'name': 'lib.model.tax.ch.notification_procedure.name',
  'code': 'MEL',
 

In [10]:

url = "https://api.bexio.com/3.0/document_templates"
response = requests.request("GET", url, headers=headers)
response.json()

[{'template_slug': '63247cba95981151f023b1d8',
  'name': 'Neue Vorlage',
  'is_default': False,
  'default_for_document_types': []},
 {'template_slug': '5e32d36544550a040b8b4588',
  'name': 'Standardvorlage',
  'is_default': True,
  'default_for_document_types': ['type_offer',
   'type_order',
   'type_invoice',
   'type_delivery',
   'type_credit_voucher',
   'type_account_statement',
   'type_article_order']}]